In [0]:
%scala
val configs = Map(
  "fs.azure.account.key.adfassignmentanjali.blob.core.windows.net" -> "VlzvQe3d8+oEN5dnPo6tW7B4B6jxNWsDyNkNcrs+CvvXl2WLsHUdWSNSFtDz06smCLIyEVVF5FMC+AStb3Wu2Q=="
)
//dbutils.fs.unmount("/mnt/bronze")
//dbutils.fs.unmount("/mnt/silver")
dbutils.fs.mount(
  source = "wasbs://bronze@adfassignmentanjali.blob.core.windows.net/",
  mountPoint = "/mnt/bronze/",
  extraConfigs = configs)

dbutils.fs.mount(
  source = "wasbs://silver@adfassignmentanjali.blob.core.windows.net/",
  mountPoint = "/mnt/silver/",
  extraConfigs = configs)




configs: scala.collection.immutable.Map[String,String] = Map(fs.azure.account.key.adfassignmentanjali.blob.core.windows.net -> VlzvQe3d8+oEN5dnPo6tW7B4B6jxNWsDyNkNcrs+CvvXl2WLsHUdWSNSFtDz06smCLIyEVVF5FMC+AStb3Wu2Q==)
res4: Boolean = true

In [0]:
#import
from pyspark.sql.functions import col
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, split, to_date, date_format
from delta.tables import *
import re

# Q1
df = spark.read.format("csv").option("header", "true").load("/mnt/bronze/customer")

# Q2
def camel_to_snake(name):
    name = name.replace(" ", "_")
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1\2', s1).lower()

# Register the function as a UDF
camel_to_snake_udf = udf(camel_to_snake, StringType())

# Apply the UDF to column names and select with new column names
df = df.select(*[col(c).alias(camel_to_snake(c)) for c in df.columns])

# Show the result
df.show()

# Q3
df = df.withColumn("first_name", split(df["name"], " ").getItem(0)) \
       .withColumn("last_name", split(df["name"], " ").getItem(1))

#Q4
df = df.withColumn("domain", split(split(df["email_id"], "@").getItem(1), "\.").getItem(0))

#Q5
df = df.withColumn("gender", when(col("gender") == "male", "M").otherwise("F"))

#Q6

df = df.withColumn("date", date_format(to_date(split(col("joining_date"), " ").getItem(0), "dd-MM-yyyy"), "yyyy-MM-dd")) \
       .withColumn("time", split(col("joining_date"), " ").getItem(1))

#Q7
df = df.withColumn("expenditure_status", when(col("spent") < 200, "MINIMUM").otherwise("MAXIMUM"))
df.show()

#Q8

# Location of the Delta table
delta_path = "dbfs:/mnt/silver/sales_view/customer/"

df.write.format("csv").mode("overwrite").save(delta_path)


+-----------+----------------+--------------------+--------------------+------+---+--------------------+----------+--------+------+------+--------------+--------------------+----------+
|customer_id|            name|            email_id|             address|gender|age|        joining_date|registered|order_id|orders| spent|           job|             hobbies|is_married|
+-----------+----------------+--------------------+--------------------+------+---+--------------------+----------+--------+------+------+--------------+--------------------+----------+
|   amxjo131|     Joseph Rice|josephrice131@sli...|91773 Miller Shoa...|  male| 43|    07-12-2022 20:15|05-05-2019|    NULL|     7|568.29|        Artist|      Playing sports|     FALSE|
|   amxga386|      Gary Moore|garymoore386@slin...|6450 John Lodge T...|  male| 71|    07-12-2021 20:15|20-05-2020|    NULL|    11|568.92|        Artist|            Swimming|      TRUE|
|    amxj944|     John Walker|johnwalker944@sli...|27265 Murray Isla..

In [0]:
delta_path = "dbfs:/mnt/silver/sales_view/customer/"

verified_df = spark.read.format("delta").load(delta_path)
verified_df.show()

from pyspark.sql import SparkSession
from delta.tables import DeltaTable

# Initialize Spark Session (Assuming it's not already started)
spark = SparkSession.builder.appName("Delete Delta Table").getOrCreate()

# Check if the Delta table exists
if DeltaTable.isDeltaTable(spark, delta_path):
    print("Delta table exists and will be deleted.")
else:
    print("Delta table does not exist at the specified path.")

dbutils.fs.rm(delta_path, recurse=True)

+--------+----------------+--------------------+--------------------+---+---+--------------------+----------+----+---+------+--------------+--------------------+-----+----------+----------+------------+----------+-----+-------+
|     _c0|             _c1|                 _c2|                 _c3|_c4|_c5|                 _c6|       _c7| _c8|_c9|  _c10|          _c11|                _c12| _c13|      _c14|      _c15|        _c16|      _c17| _c18|   _c19|
+--------+----------------+--------------------+--------------------+---+---+--------------------+----------+----+---+------+--------------+--------------------+-----+----------+----------+------------+----------+-----+-------+
|amxjo131|     Joseph Rice|josephrice131@sli...|91773 Miller Shoa...|  M| 43|    07-12-2022 20:15|05-05-2019|NULL|  7|568.29|        Artist|      Playing sports|FALSE|    Joseph|      Rice|slingacademy|2022-12-07|20:15|MAXIMUM|
|amxga386|      Gary Moore|garymoore386@slin...|6450 John Lodge T...|  M| 71|    07-12-2

True

In [0]:

from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import re

# Q1
df = spark.read.format("csv").option("header", "true").load("/mnt/bronze/products")

def camel_to_snake(name):
    name = name.replace(" ", "_")
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1\2', s1).lower()

# Register the function as a UDF
camel_to_snake_udf = udf(camel_to_snake, StringType())

# Apply the UDF to column names and select with new column names
df = df.select(*[col(c).alias(camel_to_snake(c)) for c in df.columns])

# Show the result
df.show()

#Q2
df = df.withColumn(
    "sub_category",
    when(col("category_id") == 1, "phone")
    .when(col("category_id") == 2, "laptop")
    .when(col("category_id") == 3, "playstation")
    .when(col("category_id") == 4, "e-device")
    .otherwise("unknown")  # Handling potential unknown category_ids
)
# Show the result
df.show()

# Location of the Delta table
delta_path = "dbfs:/mnt/silver/sales_view/products/"

df.write.format("delta").mode("overwrite").save(delta_path)

+------------+--------------------+------------+--------------------+-----------+------+--------------+-----------+----------+----------+--------------------+------+-----------+---------+--------+-------------+
|  product_id|        product_name|product_code|         description|category_id| price|stock_quantity|supplier_id|created_at|updated_at|           image_url|weight|expiry_date|is_active|tax_rate|     store_id|
+------------+--------------------+------------+--------------------+-----------+------+--------------+-----------+----------+----------+--------------------+------+-----------+---------+--------+-------------+
|  PROD1A23B4|     Apple iPhone 12|      IPHN12|The latest iPhone...|          1|999.99|           100|        101|01-01-2023|31-12-2023|https://example.c...|0.5 kg|       NULL|        1|       5|STORE1D2E3F4G|
| PROD5C67D8E|  Samsung Galaxy S21|       SGS21|High-performance ...|          1|899.99|           150|        102|02-01-2023|30-12-2023|https://example.c..

In [0]:

from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from pyspark.sql.functions import when, col, split, to_date, date_format
import re

# Q1
df = spark.read.format("csv").option("header", "true").load("/mnt/bronze/store")

def camel_to_snake(name):
    name = name.replace(" ", "_")
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1\2', s1).lower()

# Register the function as a UDF
camel_to_snake_udf = udf(camel_to_snake, StringType())

# Apply the UDF to column names and select with new column names
df = df.select(*[col(c).alias(camel_to_snake(c)) for c in df.columns])

# Show the result
df.show()

#Q2
df = df.withColumn("store_category", split(split(col("email_address"), "@").getItem(1), "\\.")[0])
# Show the result
df.show()

#Q3
df = df.withColumn("created_at", date_format(to_date(split(col("created_at"), " ").getItem(0), "dd-MM-yyyy"), "yyyy-MM-dd"))
df = df.withColumn("updated_at", date_format(to_date(split(col("updated_at"), " ").getItem(0), "dd-MM-yyyy"), "yyyy-MM-dd"))
# Show the result
df.show()

# Location of the Delta table
delta_path = "dbfs:/mnt/silver/sales_view/store/"

df.write.format("delta").mode("overwrite").save(delta_path)

+-------------+--------------+-------------+---------------+--------------+--------------------+------------+-------------------+----------+----------+
|     store_id|    store_name|     location|   manager_name|  phone_number|       email_address|opening_date|         store_type|created_at|updated_at|
+-------------+--------------+-------------+---------------+--------------+--------------------+------------+-------------------+----------+----------+
|STORE1D2E3F4G|      MusicHut|       Denver|   Robert White|(123) 456-7890|robertwhite@music...|       44136|Music & Instruments|11-01-2023|21-12-2023|
|STORE5H6I7J8K|  ArtisticNest|      Phoenix|    Laura Green|(456) 789-0123|lauragreen@artist...|       44166|       Art Supplies|12-01-2023|20-12-2023|
|STOREL9M0N1O2|   OutdoorLife|     Portland|    Steven Gray|(789) 012-3456|stevengray@outdoo...|       44197|  Outdoor Equipment|13-01-2023|19-12-2023|
|STORE3P4Q5R6S|     HealthHub|    Baltimore| Patricia Adams|(012) 345-6789|patriciaadams

In [0]:

from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from pyspark.sql.functions import when, col, split, to_date, date_format
import re

# Q1
df = spark.read.format("csv").option("header", "true").load("/mnt/bronze/sales")

def camel_to_snake(name):
    name = name.replace(" ", "_")
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1\2', s1).lower()

# Register the function as a UDF
camel_to_snake_udf = udf(camel_to_snake, StringType())

# Apply the UDF to column names and select with new column names
df = df.select(*[col(c).alias(camel_to_snake(c)) for c in df.columns])

# Show the result
df.show()

# Location of the Delta table
delta_path = "dbfs:/mnt/silver/sales_view/customer_sales/"

df.write.format("delta").mode("overwrite").save(delta_path)


+--------------------+---------------+--------------+-------------+----------+--------------+----------+------------+------+--------+------+-----+---------+--------------------+--------------+--------------+--------+---------+
|           orderdate|       category|          city|      country|customerid|       orderid|postalcode|  product_id|profit|quantity|region|sales|  segment|            shipdate|      shipmode|         state|latitude|longitude|
+--------------------+---------------+--------------+-------------+----------+--------------+----------+------------+------+--------+------+-----+---------+--------------------+--------------+--------------+--------+---------+
|2011-01-11T00:00:...|Office Supplies|   Springfield|United States|  amxar881|CA-2011-149020|     22153| PROD7A8B9C0|     1|       1| South|    3|Corporate|2011-01-16T00:00:...|Standard Class|      Virginia| 38.7449|  -77.237|
|2011-01-12T00:00:...|      Furniture|         Dover|United States|  amxra702|CA-2011-130092

In [0]:
delta_path = "dbfs:/mnt/silver/sales_view/customer_sales/"

verified_df = spark.read.format("csv").load(delta_path)
verified_df.show()

from pyspark.sql import SparkSession
from delta.tables import DeltaTable

# Initialize Spark Session (Assuming it's not already started)
spark = SparkSession.builder.appName("Delete Delta Table").getOrCreate()

# Check if the Delta table exists
if DeltaTable.isDeltaTable(spark, delta_path):
    print("Delta table exists and will be deleted.")
else:
    print("Delta table does not exist at the specified path.")

dbutils.fs.rm(delta_path, recurse=True)

+--------------------+---------------+--------------+-------------+--------+--------------+-----+------------+---+---+-----+----+---------+--------------------+--------------+--------------+-------+---------+
|                 _c0|            _c1|           _c2|          _c3|     _c4|           _c5|  _c6|         _c7|_c8|_c9| _c10|_c11|     _c12|                _c13|          _c14|          _c15|   _c16|     _c17|
+--------------------+---------------+--------------+-------------+--------+--------------+-----+------------+---+---+-----+----+---------+--------------------+--------------+--------------+-------+---------+
|2011-01-11T00:00:...|Office Supplies|   Springfield|United States|amxar881|CA-2011-149020|22153| PROD7A8B9C0|  1|  1|South|   3|Corporate|2011-01-16T00:00:...|Standard Class|      Virginia|38.7449|  -77.237|
|2011-01-12T00:00:...|      Furniture|         Dover|United States|amxra702|CA-2011-130092|19901|  PROD1A23B4|  3|  2| East|  10| Consumer|2011-01-15T00:00:...|   F

True

In [0]:
%scala
val configs = Map(
  "fs.azure.account.key.adfassignmentanjali.blob.core.windows.net" -> "VlzvQe3d8+oEN5dnPo6tW7B4B6jxNWsDyNkNcrs+CvvXl2WLsHUdWSNSFtDz06smCLIyEVVF5FMC+AStb3Wu2Q=="
)
dbutils.fs.mount(
  source = "wasbs://gold@adfassignmentanjali.blob.core.windows.net/",
  mountPoint = "/mnt/gold/",
  extraConfigs = configs)
  

configs: scala.collection.immutable.Map[String,String] = Map(fs.azure.account.key.adfassignmentanjali.blob.core.windows.net -> VlzvQe3d8+oEN5dnPo6tW7B4B6jxNWsDyNkNcrs+CvvXl2WLsHUdWSNSFtDz06smCLIyEVVF5FMC+AStb3Wu2Q==)
res6: Boolean = true

In [0]:
#import
from pyspark.sql.functions import col
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, split, to_date, date_format
from delta.tables import *
import re

product_path = "/mnt/silver/sales_view/products/"
store_path = "/mnt/silver/sales_view/store/"
customer_sales_path = "/mnt/silver/sales_view/customer_sales/"

# Read the tables
product_df = spark.read.format("delta").load(product_path)
store_df = spark.read.format("delta").load(store_path)
customer_sales_df = spark.read.format("delta").load(customer_sales_path)

# Q1
# Join product and store tables
product_store_df = product_df.join(
    store_df,
    product_df.store_id == store_df.store_id,
    "inner"
).select(
    store_df.store_id,
    store_df.store_name,
    store_df.location,
    store_df.manager_name,
    product_df.product_name,
    product_df.product_id,
    product_df.product_code,
    product_df.description,
    product_df.category_id,
    product_df.price,
    product_df.stock_quantity,
    product_df.supplier_id,
    product_df.created_at,
    product_df.updated_at,
    product_df.image_url,
    product_df.weight,
    product_df.expiry_date,
    product_df.is_active,
    product_df.tax_rate
)
product_store_df.show()

customer_sales_df.show()

# Assuming customer_sales_df has the necessary fields as outlined
full_df = customer_sales_df.join(
    product_store_df,
    customer_sales_df.product_id == product_store_df.product_id,  # Ensure these fields match in your schema
    "inner"
).select(
    customer_sales_df.orderdate.alias("OrderDate"),
    product_store_df.category_id.alias("Category"),
    store_df.location.alias("City"),
    customer_sales_df.customerid.alias("CustomerID"),
    customer_sales_df.orderid.alias("OrderID"),
    product_store_df.product_code.alias("ProductCode"),
    product_store_df.product_id.alias("ProductID"),
    customer_sales_df.profit.alias("Profit"),
    customer_sales_df.region.alias("Region"),
    customer_sales_df.sales.alias("Sales"),
    customer_sales_df.segment.alias("Segment"),
    customer_sales_df.shipdate.alias("ShipDate"),
    customer_sales_df.shipmode.alias("ShipMode"),
    store_df.store_name,
    store_df.location,
    store_df.manager_name,
    product_store_df.product_name,
    product_store_df.price,
    product_store_df.stock_quantity,
    product_store_df.image_url
)

full_df.show()
# Q2
gold_path = "/mnt/gold/sales_view/StoreProductSalesAnalysis/"

# Write or overwrite the existing Delta table
full_df.write.format("delta").mode("overwrite").save(gold_path)



+-------------+-------------+------------+--------------+--------------------+------------+------------+--------------------+-----------+------+--------------+-----------+----------+----------+--------------------+------+-----------+---------+--------+
|     store_id|   store_name|    location|  manager_name|        product_name|  product_id|product_code|         description|category_id| price|stock_quantity|supplier_id|created_at|updated_at|           image_url|weight|expiry_date|is_active|tax_rate|
+-------------+-------------+------------+--------------+--------------------+------------+------------+--------------------+-----------+------+--------------+-----------+----------+----------+--------------------+------+-----------+---------+--------+
|STORE1D2E3F4G|     MusicHut|      Denver|  Robert White|     Apple iPhone 12|  PROD1A23B4|      IPHN12|The latest iPhone...|          1|999.99|           100|        101|01-01-2023|31-12-2023|https://example.c...|0.5 kg|       NULL|        

In [0]:
delta_path = "dbfs:/mnt/gold/sales_view/StoreProductSalesAnalysis/"

verified_df = spark.read.format("delta").load(delta_path)
verified_df.show()

from pyspark.sql import SparkSession
from delta.tables import DeltaTable

# Initialize Spark Session (Assuming it's not already started)
spark = SparkSession.builder.appName("Delete Delta Table").getOrCreate()

# Check if the Delta table exists
if DeltaTable.isDeltaTable(spark, delta_path):
    print("Delta table exists and will be deleted.")
else:
    print("Delta table does not exist at the specified path.")

dbutils.fs.rm(delta_path, recurse=True)

+---------+--------+----+----------+-------+---------+------+------+-----+-------+--------+--------+----------+--------+------------+------------+-----+--------------+---------+
|OrderDate|Category|City|CustomerID|OrderID|ProductID|Profit|Region|Sales|Segment|ShipDate|ShipMode|store_name|location|manager_name|product_name|price|stock_quantity|image_url|
+---------+--------+----+----------+-------+---------+------+------+-----+-------+--------+--------+----------+--------+------------+------------+-----+--------------+---------+
+---------+--------+----+----------+-------+---------+------+------+-----+-------+--------+--------+----------+--------+------------+------------+-----+--------------+---------+

Delta table exists and will be deleted.


True

In [0]:
%scala
val configs = Map(
  "fs.azure.account.key.adfassignmentanjali.blob.core.windows.net" -> "VlzvQe3d8+oEN5dnPo6tW7B4B6jxNWsDyNkNcrs+CvvXl2WLsHUdWSNSFtDz06smCLIyEVVF5FMC+AStb3Wu2Q=="
)
dbutils.fs.unmount("/mnt/gold")
dbutils.fs.mount(
  source = "wasbs://silver@adfassignmentanjali.blob.core.windows.net/",
  mountPoint = "/mnt/silver/",
  extraConfigs = configs)